# 학습1 - Model 1 (번개장터 데이터로 학습)

In [ ]:
# GPU 에러 보기 위함

import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ['TORCH_USE_CUDA_DSA'] = '1'

# 환경 설정

In [ ]:
# 통합코드 (GPT)

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip3 install kobert-transformers

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_transformers import get_kobert_model, get_tokenizer

# device = torch.device("cuda:0") # GPU
# device = torch.device("cpu") # CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = get_kobert_model()
tokenizer = get_tokenizer() # vocab 역할

# 이후부터 모델 및 토크나이저를 사용하여 작업 진행.


# 데이터셋 전처리

In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/KDT/Final Project/data/sample_bj.csv')

In [ ]:
data

,id,name,cat_id
0,207604546,아쿠비클럽 숏패딩,310300200
1,172324242,듀베티카 이탈리아 브랜드 경량 패딩,310300200
2,216829622,"자라 러버라이즈드 패딩점퍼 xs 샌드(세탁1회가 전부,착용안함,새상품)",310300200
3,208445966,아이보리 퀼팅 패딩,310300200
4,214399932,Kenneth Cole 케네스콜 미들 다운패딩 S,310300200
...,...,...,...
156995,217819703,"플라워, 곰 키링, 키홀더, 열쇠고리",400999
156996,174817306,쿠로미핀,400999
156997,225929172,지방시 U 패드락,400999
156998,204684604,고양이 쿠키 브로치/페이크스위츠/로리타/카와이스타일/미니어쳐/스위티시모,400999


In [ ]:
# 데이터 전처리


In [ ]:
# 번개장터 카테고리id - 카테고리명 매칭 작업

cate = pd.read_excel('/content/drive/MyDrive/KDT/Final Project/data/categroy.xlsx', engine='openpyxl')
cate = cate.rename(columns={'대분류':'cat1', '중분류':'cat2', '소분류':'cat3'})
cate

,catid,cat1,cat2,cat3
0,310300200,여성의류,아우터,패딩
1,310300100,여성의류,아우터,점퍼
2,310300300,여성의류,아우터,코트
3,310300400,여성의류,아우터,자켓
4,310300600,여성의류,아우터,가디건
...,...,...,...,...
152,400140100,패션 액세서리,양말/스타킹,양말
153,400140200,패션 액세서리,양말/스타킹,스타킹
154,400090,패션 액세서리,우산/양산,NaN
155,400600,패션 액세서리,키링/키케이스,NaN


In [ ]:
cate.reset_index(inplace=True)
cate.rename(columns={'index': 'cate_enco'}, inplace=True)
cate

,cate_enco,catid,cat1,cat2,cat3
0,0,310300200,여성의류,아우터,패딩
1,1,310300100,여성의류,아우터,점퍼
2,2,310300300,여성의류,아우터,코트
3,3,310300400,여성의류,아우터,자켓
4,4,310300600,여성의류,아우터,가디건
...,...,...,...,...,...
152,152,400140100,패션 액세서리,양말/스타킹,양말
153,153,400140200,패션 액세서리,양말/스타킹,스타킹
154,154,400090,패션 액세서리,우산/양산,NaN
155,155,400600,패션 액세서리,키링/키케이스,NaN


In [ ]:
# 원래 카테고리id와 라벨인코딩한 카테고리id 저장해놓은 딕셔너리

cate_num = {}
for idx, row in cate.iterrows():
    cate_num[row['catid']] = row['cate_enco']
cate_num

{310300200: 0,
 310300100: 1,
 310300300: 2,
 310300400: 3,
 310300600: 4,
 310300500: 5,
 310260800: 6,
 310260700: 7,
 310260600: 8,
 310260500: 9,
 310260400: 10,
 310260100: 11,
 310260200: 12,
 310260300: 13,
 310150080: 14,
 310150010: 15,
 310150030: 16,
 310150090: 17,
 310150040: 18,
 310150070: 19,
 310150999: 20,
 310130030: 21,
 310130080: 22,
 310130040: 23,
 310120030: 24,
 310120110: 25,
 310120020: 26,
 310250: 27,
 310400100: 28,
 310400200: 29,
 310400999: 30,
 310200200: 31,
 310200100: 32,
 310220300: 33,
 310220100: 34,
 310220200: 35,
 310220998: 36,
 320300300: 37,
 320300200: 38,
 320300100: 39,
 320300400: 40,
 320300600: 41,
 320300500: 42,
 320210600: 43,
 320210500: 44,
 320210700: 45,
 320210400: 46,
 320210100: 47,
 320210200: 48,
 320210300: 49,
 320120600: 50,
 320120100: 51,
 320120200: 52,
 320120300: 53,
 320120700: 54,
 320120999: 55,
 320400: 56,
 320500100: 57,
 320500200: 58,
 320500999: 59,
 320160700: 60,
 320160800: 61,
 320180700: 62,
 3201806

In [ ]:
# 카테고리 df 빈 값 채워주기

cate = cate.fillna('None')
cate

,cate_enco,catid,cat1,cat2,cat3
0,0,310300200,여성의류,아우터,패딩
1,1,310300100,여성의류,아우터,점퍼
2,2,310300300,여성의류,아우터,코트
3,3,310300400,여성의류,아우터,자켓
4,4,310300600,여성의류,아우터,가디건
...,...,...,...,...,...
152,152,400140100,패션 액세서리,양말/스타킹,양말
153,153,400140200,패션 액세서리,양말/스타킹,스타킹
154,154,400090,패션 액세서리,우산/양산,None
155,155,400600,패션 액세서리,키링/키케이스,None


In [ ]:
# 라벨인코딩한 카테고리id와 카테고리명 저장해놓은 딕셔너리

cate_dict = {}

for idx,row in cate.iterrows():
    if row['cat2'] == 'None':
        cate_dict[row.cate_enco] = row.cat1
    elif row['cat3'] == 'None':
        cate_dict[row.cate_enco] = row.cat1+' - '+row.cat2
    else:
        cate_dict[row.cate_enco] = row.cat1+' - '+row.cat2+' - '+row.cat3

cate_dict

{0: '여성의류 - 아우터 - 패딩',
 1: '여성의류 - 아우터 - 점퍼',
 2: '여성의류 - 아우터 - 코트',
 3: '여성의류 - 아우터 - 자켓',
 4: '여성의류 - 아우터 - 가디건',
 5: '여성의류 - 아우터 - 조끼/베스트',
 6: '여성의류 - 상의 - 니트/스웨터',
 7: '여성의류 - 상의 - 후드티/후드집업',
 8: '여성의류 - 상의 - 맨투맨',
 9: '여성의류 - 상의 - 블라우스',
 10: '여성의류 - 상의 - 셔츠',
 11: '여성의류 - 상의 - 반팔 티셔츠',
 12: '여성의류 - 상의 - 긴팔 티셔츠',
 13: '여성의류 - 상의 - 민소매 티셔츠',
 14: '여성의류 - 바지 - 데님/청바지',
 15: '여성의류 - 바지 - 슬랙스',
 16: '여성의류 - 바지 - 면바지',
 17: '여성의류 - 바지 - 반바지',
 18: '여성의류 - 바지 - 트레이닝/조거팬츠',
 19: '여성의류 - 바지 - 레깅스',
 20: '여성의류 - 바지 - 기타 바지',
 21: '여성의류 - 치마 - 롱 스커트',
 22: '여성의류 - 치마 - 미디 스커트',
 23: '여성의류 - 치마 - 미니 스커트',
 24: '여성의류 - 원피스 - 롱 원피스',
 25: '여성의류 - 원피스 - 미디 원피스',
 26: '여성의류 - 원피스 - 미니 원피스',
 27: '여성의류 - 점프수트',
 28: '여성의류 - 셋업/세트 - 정장/셋업',
 29: '여성의류 - 셋업/세트 - 트레이닝/스웨트 셋업',
 30: '여성의류 - 셋업/세트 - 기타 셋업/세트',
 31: '여성의류 - 언더웨어/홈웨어 - 홈웨어',
 32: '여성의류 - 언더웨어/홈웨어 - 언더웨어',
 33: '여성의류 - 테마/이벤트 - 코스튬/코스프레',
 34: '여성의류 - 테마/이벤트 - 한복',
 35: '여성의류 - 테마/이벤트 - 드레스',
 36: '여성의류 - 테마/이벤트 - 기타 테마/이벤트',
 37: '남성의류

In [ ]:
# 현재 데이터
data

,id,name,cat_id
0,207604546,아쿠비클럽 숏패딩,310300200
1,172324242,듀베티카 이탈리아 브랜드 경량 패딩,310300200
2,216829622,"자라 러버라이즈드 패딩점퍼 xs 샌드(세탁1회가 전부,착용안함,새상품)",310300200
3,208445966,아이보리 퀼팅 패딩,310300200
4,214399932,Kenneth Cole 케네스콜 미들 다운패딩 S,310300200
...,...,...,...
156995,217819703,"플라워, 곰 키링, 키홀더, 열쇠고리",400999
156996,174817306,쿠로미핀,400999
156997,225929172,지방시 U 패드락,400999
156998,204684604,고양이 쿠키 브로치/페이크스위츠/로리타/카와이스타일/미니어쳐/스위티시모,400999


In [ ]:
data['cate_enco'] = 0
for idx, row in data.iterrows():
    label_name = cate_num[row['cat_id']]
    data.iloc[idx, 3] = label_name
data = data.drop(['cat_id'], axis=1)
data

,id,name,cate_enco
0,207604546,아쿠비클럽 숏패딩,0
1,172324242,듀베티카 이탈리아 브랜드 경량 패딩,0
2,216829622,"자라 러버라이즈드 패딩점퍼 xs 샌드(세탁1회가 전부,착용안함,새상품)",0
3,208445966,아이보리 퀼팅 패딩,0
4,214399932,Kenneth Cole 케네스콜 미들 다운패딩 S,0
...,...,...,...
156995,217819703,"플라워, 곰 키링, 키홀더, 열쇠고리",156
156996,174817306,쿠로미핀,156
156997,225929172,지방시 U 패드락,156
156998,204684604,고양이 쿠키 브로치/페이크스위츠/로리타/카와이스타일/미니어쳐/스위티시모,156


In [ ]:
len(data['cate_enco'].unique())

157

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157000 entries, 0 to 156999
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         157000 non-null  int64 
 1   name       157000 non-null  object
 2   cate_enco  157000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [ ]:
data_list = []
for q, label in zip(data['name'], data['cate_enco']):
    data1 = []
    data1.append(q)
    data1.append(label)

    data_list.append(data1)

In [ ]:
data_list[0] # 이전 코드로 확인

['노스페이스 롱패딩', 310090050]

In [ ]:
data_list[0] # 현재 코드

['아쿠비클럽 숏패딩', 0]

# Train data & Test data

In [ ]:
# train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=25, stratify=[data[1] for data in data_list])

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

125600
31400


# KoBERT 입력 데이터로 만들기

In [ ]:
from transformers import BertTokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        self.sentences = [i[sent_idx] for i in dataset]
        self.labels = [torch.tensor(i[label_idx], dtype=torch.long) for i in dataset]
        self.tokenizer = bert_tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair

    def __getitem__(self, i):
        sentence = self.sentences[i]
        label = self.labels[i]

        inputs = self.tokenizer.encode_plus(
            sentence,
            None if not self.pair else sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length' if self.pad else None,
            truncation=True
        )

        input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
        valid_length = torch.tensor(len(inputs['input_ids']), dtype=torch.long)

        return (input_ids, attention_mask, valid_length, label)

    def __len__(self):
        return len(self.labels)


In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5      # 5일 때 정확도 train: 0.79, test: 0.74 - 1시간 52분?
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# 토큰화
tokenizer = get_tokenizer()
tok = tokenizer

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
data_train[0]

(tensor([   2,  517, 7563, 6449, 6599, 7681,  517, 7861, 6191, 5920, 1174, 5390,
         7096, 3133, 6579, 6493, 6122,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(64),
 tensor(117))

In [ ]:
# 이전 코드로 확인
dataset_train[0] # [CLS], "화이트", "##무드", "중청", "##바", "##지", [SEP]

['화이트무드 중청바지', 310140030]

In [ ]:
dataset_train[0]

['큐빅셋팅 핸드메이드 귀걸이 악세사리', 117]

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

# KoBERT 학습모델 만들기

In [ ]:
len(data['cate_enco'].unique()) # 카테고리 수 157개

157

In [ ]:
class BERTClassifier(nn.Module): # for 번개장터 (메인)
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 157,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert                # pretrained_bert 모델
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate is not None:
            self.dropout = nn.Dropout(p=dr_rate)
        else:
            self.dropout = None

    def forward(self, input_ids, attention_mask, valid_length):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    return_dict=False)
        output = self.dropout(pooled_output)
        output = self.classifier(output)

        return output


In [ ]:
# GPU

from torch.optim import AdamW

# BERT 모델 불러오기
kobert_model = get_kobert_model()
model = BERTClassifier(kobert_model, dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
optimizer = AdamW(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss().to(device)

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [ ]:
## CPU

# BERT 모델 불러오기
# kobert_model = get_kobert_model()
# model = BERTClassifier(kobert_model,  dr_rate=0.5)
# # optimizer와 schedule 설정
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss()

# t_total = len(train_dataloader) * num_epochs
# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# # 정확도 측정을 위한 함수 정의
# def calc_accuracy(X, Y):
#     max_vals, max_indices = torch.max(X, 1)
#     train_acc = (max_indices.cpu() == Y.cpu()).sum().data.numpy() / max_indices.size()[0]
#     return train_acc


# KoBERT 모델 학습시키기

In [ ]:
# 테스트 데이터로 에폭5 해서 구글드라이브에 저장 완료

from tqdm.notebook import tqdm

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.to(device)
        attention_mask = attention_mask.to(device)
        valid_length = valid_length.to(device)
        label = label.to(device)
        out = model(token_ids, attention_mask, valid_length)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        valid_length = valid_length.to(device)
        label = label.long().to(device)
        out = model(token_ids, attention_mask, valid_length)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


  0%|          | 0/1963 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 5.1522674560546875 train acc 0.0
epoch 1 batch id 201 loss 5.049585819244385 train acc 0.007695895522388059
epoch 1 batch id 401 loss 4.855753421783447 train acc 0.013326059850374064
epoch 1 batch id 601 loss 3.906931161880493 train acc 0.06304596505823627
epoch 1 batch id 801 loss 3.0955638885498047 train acc 0.128667290886392
epoch 1 batch id 1001 loss 2.5461056232452393 train acc 0.18111575924075923
epoch 1 batch id 1201 loss 1.6647093296051025 train acc 0.22377185678601166
epoch 1 batch id 1401 loss 1.6645587682724 train acc 0.261978051391863
epoch 1 batch id 1601 loss 1.6351481676101685 train acc 0.29404473766396
epoch 1 batch id 1801 loss 0.9929189682006836 train acc 0.32202595780122156
epoch 1 train acc 0.3421978476821192


  0%|          | 0/491 [00:00<?, ?it/s]

epoch 1 test acc 0.5958057535641548


  0%|          | 0/1963 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7391038696537677


  0%|          | 0/1963 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.8121553659439087 train acc 0.765625
epoch 5 batch id 201 loss 1.1581765413284302 train acc 0.7838930348258707
epoch 5 batch id 401 loss 0.7130153179168701 train acc 0.7913809226932669
epoch 5 batch id 601 loss 0.4865720570087433 train acc 0.7926632695507487
epoch 5 batch id 801 loss 0.6131734251976013 train acc 0.7932662297128589
epoch 5 batch id 1001 loss 0.8631790280342102 train acc 0.7952984515484516
epoch 5 batch id 1201 loss 0.47616538405418396 train acc 0.7961464404662781
epoch 5 batch id 1401 loss 0.6653780937194824 train acc 0.7972764989293362
epoch 5 batch id 1601 loss 0.8098092079162598 train acc 0.7975288881948782
epoch 5 batch id 1801 loss 0.4835813641548157 train acc 0.7981763603553581
epoch 5 train acc 0.7988490193581254


  0%|          | 0/491 [00:00<?, ?it/s]

epoch 5 test acc 0.7430371690427698


In [ ]:
# 모델 저장 경로 지정
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model.pth"

# 모델 저장
torch.save(model.state_dict(), save_path)

In [ ]:
# 저장된 모델 불러오기

## GPU
model = BERTClassifier(kobert_model, dr_rate=0.5).to(device)
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model_kobert5.pth"
model.load_state_dict(torch.load(save_path))


## CPU
# device = torch.device("cpu")
# model = BERTClassifier(kobert_model, dr_rate=0.5)
# save_path = "/content/drive/MyDrive/KDT/Final Project/modeling/model_kobert_epoch5.pth"
# model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))


<All keys matched successfully>

# 새로운 문장 테스트

In [ ]:
# 새 코드

import numpy as np
from kobert_transformers import get_tokenizer

tokenizer = get_tokenizer()
tok = get_tokenizer()

# 카테고리와 인덱스 매핑
# category_map = {}
# for i in range(174):
#     category_map[i] = f"카테고리{i+1}"

def predict(predict_sentence):
    category_id = 0  # 예측할 카테고리의 인덱스 값
    data = [(predict_sentence, category_id)]  # 상품명, 카테고리id
    dataset_another = BERTDataset(data, sent_idx=0, label_idx=1, bert_tokenizer=tok, max_len=max_len, pad=True, pair=False)
    test_dataloader = torch.utils.data.DataLoader(dataset_another, batch_size=1, num_workers=0)  # 배치 크기를 1로 설정

    model.eval()

    for batch_id, (input_ids, attention_mask, valid_length, label) in enumerate(test_dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        valid_length = valid_length.to(device)
        label = label.to(device)

        out = model(input_ids, attention_mask, valid_length)

        test_eval = []
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) in cate_dict:
                category = cate_dict[np.argmax(logits)]
                test_eval.append(category)

        if test_eval:
            print(">> 입력하신 상품명으로부터 예측된 카테고리:", test_eval[0])
        else:
            print(">> 카테고리를 예측할 수 없습니다.")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


# 학습1에서 입력해보기!!

In [ ]:
# 질문 무한 반복! 0 입력시 종료
while True:
    sentence = input("상품명을 입력해주세요 : ")
    if sentence == '0':
        break
    predict(sentence)
    print("\n")

상품명을 입력해주세요 : 숏패딩
>> 입력하신 상품명으로부터 예측된 카테고리: 여성의류 - 아우터 - 패딩


상품명을 입력해주세요 : 시계
>> 입력하신 상품명으로부터 예측된 카테고리: 시계 - 남성시계 - 일반 시계(기타 밴드)


상품명을 입력해주세요 : 은팔찌
>> 입력하신 상품명으로부터 예측된 카테고리: 쥬얼리 - 팔찌 - 은팔찌


상품명을 입력해주세요 : 야구화
>> 입력하신 상품명으로부터 예측된 카테고리: 신발 - 스포츠화 - 야구화


상품명을 입력해주세요 : 구두
>> 입력하신 상품명으로부터 예측된 카테고리: 신발 - 여성화 - 구두


상품명을 입력해주세요 : 귀여운 피어싱
>> 입력하신 상품명으로부터 예측된 카테고리: 쥬얼리 - 귀걸이/피어싱 - 피어싱


상품명을 입력해주세요 : 예쁜 모자
>> 입력하신 상품명으로부터 예측된 카테고리: 패션 액세서리 - 모자 - 기타(모자)


상품명을 입력해주세요 : 0


# 학습2 - Model 1 + Dense Layer (네이버 쇼핑 데이터로 학습) 가중치 튜닝

In [ ]:
# 네이버 데이터

# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
nvdata = pd.read_csv('/content/drive/MyDrive/KDT/Final Project/data/nvdata_sample.csv')
nvdata

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,name,cat_id,image_url,low_price
0,84208236177,쉬폰나시 여름 정장 이너 민소매블라우스,310040070,https://shopping-phinf.pstatic.net/main_842082...,16400
1,37442049039,나시 블라우스 RMBNC24R01,310040070,https://shopping-phinf.pstatic.net/main_374420...,9000
2,83162292374,고퀄리티 여자 나시 세미 정장 이너 쉬폰 민소매 블라우스,310040070,https://shopping-phinf.pstatic.net/main_831622...,24900
3,81231777789,여성블라우스 여자면접정장블라우스 민소매나시 흰색블라우스 쉬폰 라운드,310040070,https://shopping-phinf.pstatic.net/main_812317...,26100
4,14843253654,민소매 블라우스 슬리브리스 이너웨어 여성 나시 브이넥,310040070,https://shopping-phinf.pstatic.net/main_148432...,7300
...,...,...,...,...,...
49995,39870493838,체크셔츠 CALLIMLLE 정부측 권한을 부여함 스타일리쉬 여성 2453979496,310050030,https://shopping-phinf.pstatic.net/main_398704...,192020
49996,37554623218,나바바 여성 모던 빈티지 체크 쉬폰 셔츠 NB941,310050030,https://shopping-phinf.pstatic.net/main_375546...,20490
49997,37552681303,나바바 여성 레트로 캐주얼 체크 롱셔츠 NB1003,310050030,https://shopping-phinf.pstatic.net/main_375526...,19970
49998,39854813910,여성 캔퍼스룩 포켓 데일리 루즈 체크 셔츠 남방,310050030,https://shopping-phinf.pstatic.net/main_398548...,29290


## 입력데이터로 만들기

In [ ]:
# 배치 단위로 나누기

# 입력 데이터 형태로 고치기

nvdata = nvdata.drop(['image_url','low_price'], axis=1)

nvdata['cate_enco'] = 0
for idx, row in nvdata.iterrows():
    label_name = cate_num[row['cat_id']]
    nvdata.iloc[idx, 3] = label_name
nvdata = nvdata.drop(['cat_id'], axis=1)

nvdata_list = []
for q, label in zip(nvdata['name'], nvdata['cate_enco']):
    data1 = []
    data1.append(q)
    data1.append(label)

    nvdata_list.append(data1)

# train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(nvdata_list, test_size=0.2, random_state=25, stratify=[data[1] for data in nvdata_list])

from transformers import BertTokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        self.sentences = [i[sent_idx] for i in dataset]
        self.labels = [torch.tensor(i[label_idx], dtype=torch.long) for i in dataset]
        self.tokenizer = bert_tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair

    def __getitem__(self, i):
        sentence = self.sentences[i]
        label = self.labels[i]

        inputs = self.tokenizer.encode_plus(
            sentence,
            None if not self.pair else sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length' if self.pad else None,
            truncation=True
        )

        input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
        valid_length = torch.tensor(len(inputs['input_ids']), dtype=torch.long)

        return (input_ids, attention_mask, valid_length, label)

    def __len__(self):
        return len(self.labels)

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5      # 5일 때 정확도 train: 0.61, test: 0.60 - 42분  ## 3일 때 정확도 train: 0.47, test: 0.51 - 17분
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

# 토큰화
tokenizer = get_tokenizer()
tok = tokenizer

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


## 학습1 BERT 모델 불러오기

In [ ]:
# 학습1 모델 로드

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 146,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate is not None:
            self.dropout = nn.Dropout(p=dr_rate)
        else:
            self.dropout = None

    def forward(self, input_ids, attention_mask, valid_length):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    return_dict=False)
        output = self.dropout(pooled_output)
        output = self.classifier(output)

        return output

# BERT 모델 불러오기
kobert_model = get_kobert_model()

# 저장된 모델 불러오기

## GPU
model = BERTClassifier(kobert_model, dr_rate=0.5).to(device)
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model_kobert_epoch5.pth"

model.load_state_dict(torch.load(save_path, map_location=device))


RuntimeError: ignored

In [ ]:
model

## 학습2 모델 생성

In [ ]:
# BERT 모델의 아키텍처 뒤에 Dense 레이어 추가 (학습2 모델 생성)

class BERTClassifierforNaver(nn.Module):  # 네이버 미세조정을 위한 모델
    def __init__(self,
                  bert,
                  hidden_size = 146, # 번개장터 클래스 개수 (output이 146으로 나오니까)
                  num_classes = 5,   # 네이버 카테고리 개수
                  dr_rate=None,
                  params=None):
            super(BERTClassifierforNaver, self).__init__()
            self.bert = bert  # 위에서 학습시킨 번개장터 모델
            self.dr_rate = dr_rate

            self.classifier = nn.Linear(hidden_size, num_classes)
            if dr_rate is not None:
                self.dropout = nn.Dropout(p=dr_rate)
            else:
                self.dropout = None

    def forward(self, input_ids, attention_mask, valid_length):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    return_dict=False)
        output = self.dropout(pooled_output)
        output = self.classifier(output)

        return output

    # def __init__(self, bert, hidden_size, dropout_rate=0.5, num_classes=5):
    #     super(BERTWithDenseClassifier, self).__init__(bert, hidden_size)
    #     self.dropout = nn.Dropout(dropout_rate) # self.dropout 초기화
    #     self.dense_layer = nn.Linear(hidden_size, num_classes) # num_classes: 카테고리 수

    # def forward(self, input_ids, attention_mask, valid_length):
    #     # BERT 모델의 forward 메서드 호출

    #     output = super(BERTWithDenseClassifier, self).forward(input_ids, attention_mask, valid_length)
    #     print(output)
    #     print(type(output))

    #     # Dense 레이어 적용
    #     logits = self.dense_layer(output)
    #     return logits

# GPU

from torch.optim import AdamW

# 학습1 모델 불러오기
model = BERTClassifierforNaver(model, dr_rate=0.5)


# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [ ]:
device

device(type='cuda', index=0)


## 가중치 튜닝 모델 학습

In [ ]:
# 가중치 튜닝 학습

from tqdm.notebook import tqdm

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.to(device)
        attention_mask = attention_mask.to(device)
        valid_length = valid_length.to(device)
        label = label.to(device)
        out = model(token_ids, attention_mask, valid_length)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        valid_length = valid_length.to(device)
        label = label.long().to(device)
        out = model(token_ids, attention_mask, valid_length)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


  0%|          | 0/714 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
# # 데이터 배치를 모델에 입력으로 주고, 출력을 얻음

# # 모델 생성 및 입력 데이터 전달
# model = BERTWithDenseClassifier(kobert_model, 768)  # BERTWithDenseClassifier 클래스로 모델 생성
# model.to(device)  # GPU로 모델 이동

# print(len(nvtrain_dataloader))
# for batch in nvtrain_dataloader:
#     # print(type(batch))
#     # print(len(batch))
#     # print('첫번째!!: ', batch[0])
#     # print('두번째!!: ',batch[1])
#     # print('세번째!!: ',batch[2])
#     # print('네번째!!: ',batch[3])

#     input_ids = batch[0].to(device)
#     attention_mask = batch[1].to(device)
#     valid_length = batch[2].to(device)
#     labels = batch[3].to(device)

#     # 예측값 계산
#     logits = model.forward(input_ids, attention_mask, valid_length)

#     # 손실 계산
#     loss = loss_function(logits, labels)

#     # 역전파 수행 및 모델 업데이트
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()


782


OutOfMemoryError: ignored

In [ ]:
# 출력을 손실 함수와 비교하여 가중치 튜닝을 위한 학습 진행



In [ ]:
nvdata

,id,name,cate_enco
0,84208236177,쉬폰나시 여름 정장 이너 민소매블라우스,20
1,37442049039,나시 블라우스 RMBNC24R01,20
2,83162292374,고퀄리티 여자 나시 세미 정장 이너 쉬폰 민소매 블라우스,20
3,81231777789,여성블라우스 여자면접정장블라우스 민소매나시 흰색블라우스 쉬폰 라운드,20
4,14843253654,민소매 블라우스 슬리브리스 이너웨어 여성 나시 브이넥,20
...,...,...,...
49995,39870493838,체크셔츠 CALLIMLLE 정부측 권한을 부여함 스타일리쉬 여성 2453979496,25
49996,37554623218,나바바 여성 모던 빈티지 체크 쉬폰 셔츠 NB941,25
49997,37552681303,나바바 여성 레트로 캐주얼 체크 롱셔츠 NB1003,25
49998,39854813910,여성 캔퍼스룩 포켓 데일리 루즈 체크 셔츠 남방,25


In [ ]:
len(nvdata['cate_enco'].unique())

5